In [1]:
# Make sure to also output the intermediary steps
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Auto-Cleaning Dirty Data: the Data Encoding Bot

[2IMM00] Seminar Data Mining
<br>
Angelo Majoor - 1030843
<br>
A.R.Majoor@student.tue.nl

Supervisor: dr. ir. J. (Joaquin) Vanschoren

Eindhoven University of Technology
<br>
Department of Mathematics and Computer Science
<br>
Data Mining Research Group

In [2]:
# Import all relevant libraries
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random

from nltk.corpus import wordnet as wn

import openml as oml
oml.config.apikey = '2d4efc0fbf4c75a890be14297c5ec1e4'
import sklearn

### The three different steps

As stated in the report, the process of creating the data encoding bot consists of three different steps:

 - Auto-detecting data types per feature (column)
 - Auto-detecting numeric, ordinal, categorical (integer) features
 - Auto-selecting encoding techniques for all features

## Step 0: Import the Data

In [153]:
#raw_data = pd.read_csv("ENTER_YOUR_CSV_FILE_NAME_HERE")
raw_data = pd.read_csv("Data/wine_mag.csv")
raw_data.head()
raw_data.dtypes

#raw_energy = pd.read_csv("Data/energy_train_copy.csv")
#raw_energy.head()
#raw_energy.dtypes

#raw_weather = pd.read_csv("Data/weather_train_copy.csv")
#raw_weather.head()
#raw_weather.dtypes

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude


Unnamed: 0       int64
country         object
description     object
designation     object
points           int64
price          float64
province        object
region_1        object
region_2        object
variety         object
winery          object
dtype: object

## Step 1: Auto-Detecting Data Types

In [157]:
# Function to automatically infer data types for a specific feature that has the standard 'object' data type
# Data types that we want to infer: boolean, date, float, integer, string
# Note that every feature that is not either a boolean, a date, a float or an integer, is classified as a string
# Input: Pandas Dataframe consisting of one single feature (so n*1 in size)
# Output: Data type of the feature (in string format)

def autoInferObject(raw_data_feature):
    dataType = ""
    types = ["date","float64","int64","string"] #Data types
    weights = [0,0,0,0] #Weights corresponding to the data types
    numberOfIndices = 100 # umber of different values to check in a feature
    
    featureLength = len(raw_data_feature) #Number of rows in the feature
    
    randomIndices = random.sample(range(0,featureLength), min(numberOfIndices,featureLength)) #Array of random indices
    
    # If the feature only contains two different unique values, then infer it as boolean
    if len(pd.unique(raw_data_feature)) == 2:
        dataType = "bool"
    else:
        for i in randomIndices:
            try:
                if (len(raw_data_feature[i]) <= 10 
                    and ((raw_data_feature[i][2:3] == ('-' or '/') and raw_data_feature[i][5:6] == ('-' or '/')) or 
                    (raw_data_feature[i][4:5] == ('-' or '/') and raw_data_feature[i][7:8] == ('-' or '/')))):
                    weights[0] += 1 #Date
                else:
                    weights[3] += 1 #String
            except (TypeError,ValueError,IndexError):
                try:
                    int(raw_data_feature[i])
                    if ('.' in str(raw_data_feature[i])):
                        weights[1] += 1 #Float
                    else:
                        weights[2] += 1 #Integer
                except (TypeError,ValueError,IndexError):
                    weights[3] += 1 #String
    
        #print ("Date: {}, Float64: {}, Int64: {}, String: {}".format(weights[0],weights[1],weights[2],weights[3])) #For debugging purposes
        dataType = types[weights.index(max(weights))]
        
    return dataType

In [158]:
# Function to automatically infer data types for every single feature in a raw data set
# Input: Pandas Dataframe created directly from the raw data with the pd.read_csv function
# Output: List of data types, one data type for each feature

def autoDetectDataTypes(raw_data):
    result = []
    
    for column in raw_data:
        if raw_data.dtypes[column] == "object":
            #print ("Trying to automatically infer the data type of the",column,"feature...") #For debugging purposes
            inferredType = autoInferObject(raw_data[column])
            result.append(inferredType)
            #print ("Result:",inferredType) #For debugging purposes
            # Auto-infer in step 1
        elif raw_data.dtypes[column] == "int64":
            result.append("int64")
            # Go to step 2
        else:
            # The only remaining data type is 'float64', which needs no special treatment
            result.append("float64")
        
    return result

In [181]:
predicted = autoDetectDataTypes(raw_data)
print ("\nPredicted data types:\n",predicted)


Predicted data types:
 ['int64', 'string', 'string', 'string', 'int64', 'float64', 'string', 'string', 'string', 'string', 'string']


### Testing the accuracy of the implemented solution

In [162]:
# Manually check for the ground truth, since a dirty data set has no ground truth included

#ground_truth_energy = ['date', 'string', 'int64', 'float64', 'float64', 'int64', 'float64']
#ground_truth_weather = ['string', 'float64', 'float64', 'float64', 'float64', 'string', 'float64', 'float64', 'float64']
#ground_truth_TED = ['int64', 'string', 'int64', 'string', 'int64', 'int64', 'string', 'string', 'int64', 'int64', 'string', 'string', 'string', 'string', 'string', 'string', 'int64']
#ground_truth_wine = ['int64', 'string', 'string', 'string', 'int64', 'float64', 'string', 'string', 'string', 'string', 'string']

ground_truth

['int64',
 'string',
 'string',
 'string',
 'int64',
 'float64',
 'string',
 'string',
 'string',
 'string',
 'string']

In [163]:
# Function to calculate an accuracy score for the implemented solution
# Input: Array containing the (self-made) ground truth
#        Array containing the predicted data types
# Output: Accuracy score based on the number of correct predictions

def score(ground_truth, predicted):
    correctPredictions = 0
    
    for i in range(0,len(ground_truth)):
        if ground_truth[i] == predicted[i]:
            correctPredictions += 1
        
    return correctPredictions / len(ground_truth)

In [164]:
print ("Accuracy:",score(ground_truth, predicted)*100,"%")

Accuracy: 100.0 %


### Results

In [165]:
scoreResults = [("Solar Panel Energy Production Eindhoven",100.0,"https://canvas.tue.nl/files/508283"), \
                ("Weather Measurements Eindhoven Airport",100.0,"https://canvas.tue.nl/files/508283"), \
                ("TED Talks",100.0,"https://www.kaggle.com/rounakbanik/ted-talks"), \
                ("Wine Reviews",100.0,"https://www.kaggle.com/zynicide/wine-reviews")]
pd.DataFrame(scoreResults, columns=["Data Set", "Accuracy", "Retrieved From"])

,Data Set,Accuracy,Retrieved From
0,Solar Panel Energy Production Eindhoven,100.0,https://canvas.tue.nl/files/508283
1,Weather Measurements Eindhoven Airport,100.0,https://canvas.tue.nl/files/508283
2,TED Talks,100.0,https://www.kaggle.com/rounakbanik/ted-talks
3,Wine Reviews,100.0,https://www.kaggle.com/zynicide/wine-reviews


### Flaws

 - Whenever a feature is recognized as integer by the read_csv() function, this feature cannot be a boolean anymore. This can simply be solved by adding a check for the number of unique elements in the 'autoDetectDataTypes()' function, in the 'elif' branch (where the function checks for int64's).
 - Dates that are represented as integers cannot be detected, since it is unknown whether a value represents a date, or an actual number.

## Step 1 Completed --------------------------------------------------

## Step 2: Numeric, Categorical or Ordinal Data?

In [47]:
# Load the OpenML100 data set
benchmark_suite = oml.study.get_study('OpenML100','tasks')

list_task_ids = []

# Find all task_ids
for task_id in benchmark_suite.tasks:
    list_task_ids.append(task_id)
    task = oml.tasks.get_task(task_id)
    print (task.get_dataset().name)

#list_task_ids

kr-vs-kp
letter
balance-scale
mfeat-factors
mfeat-fourier
breast-w
mfeat-karhunen
mfeat-morphological
mfeat-pixel
car
mfeat-zernike
cmc
mushroom
optdigits
credit-approval
credit-g
pendigits
segment
diabetes
soybean
spambase
splice
tic-tac-toe
vehicle
waveform-5000
electricity
satimage
eucalyptus
sick
vowel
isolet
scene
monks-problems-1
monks-problems-2
monks-problems-3
JapaneseVowels
synthetic_control
irish
analcatdata_authorship
analcatdata_dmft
profb
collins
mnist_784
sylva_agnostic
gina_agnostic
ada_agnostic
mozilla4
pc4
pc3
jm1
kc2
kc1
pc1
KDDCup09_churn
KDDCup09_upselling
MagicTelescope
adult
wilt
wdbc
micro-mass
phoneme
one-hundred-plants-margin
one-hundred-plants-shape
one-hundred-plants-texture
qsar-biodeg
wall-robot-navigation
semeion
steel-plates-fault
tamilnadu-electricity
hill-valley
ilpd
madelon
nomao
ozone-level-8hr
cardiotocography
climate-model-simulation-crashes
cnae-9
eeg-eye-state
first-order-theorem-proving
gas-drift
banknote-authentication
blood-transfusion-service

In [53]:
trial = oml.tasks.get_task(list_task_ids[3])
print (trial.get_dataset().name)

X, y = trial.get_X_and_y()
for i in range (0,5):
    print (X[i])
    print (y[i])

mfeat-factors
[   98.   236.   531.   673.   607.   647.     2.     9.     3.     6.
     8.     5.   225.   517.   652.   624.   628.   994.     7.    22.
    28.    13.    10.    19.   305.   481.   667.   663.  1009.   727.
    38.    28.    18.    11.    20.    10.   287.   567.   651.   742.
   824.   900.    26.    34.    30.     8.    16.    13.   248.   556.
   631.   796.   926.   748.    39.    34.    18.     9.    17.    12.
   248.   540.   506.   814.  1051.   728.    38.    28.     5.    13.
    16.     8.   246.   518.   751.   579.   699.  1062.    13.    30.
    28.    10.    16.    16.   276.   344.   682.   500.   709.   916.
    10.    30.    23.    17.    15.    14.   357.   435.   829.   610.
   745.   994.    20.     7.    24.    12.    10.     9.   355.   409.
   477.   886.   976.   723.    30.    24.    14.     7.     7.     8.
   290.   352.   435.   753.   894.   751.    29.    29.     2.    13.
    13.    14.   260.   286.   562.   698.   665.   757.    11.

In [170]:
# Function to check if a feature contains categorical data, specifically concerning strings
# If a feature contains at most 25 unique elements, this feature is always considered as categorical
# If a feature contains between 26 and k (where k is user specified) unique values, then the function 
# calculates a similarity score between all of the different values. If this score is higher than 0.70 
# a feature is also considered to be categorical (since the values at least have some relationship).
# Input: Pandas Dataframe consisting of one single feature (so n*1 in size)
#        A user-determined value k (the critical value: more than k unique values cannot be categorical)
# Output: A boolean stating whether the supplied feature is categorical or not

def autoCheckCategoricalString(raw_data_feature, k):
    categorical = False
    
    allWords = pd.unique(raw_data_feature) #All unique strings in the feature
    similarityScores = []
    
    if (len(allWords) <= 25): #If there are less than 25 unique strings, it is categorical
        categorical = True
        
    elif (len(allWords) <= k): #Else if there are less than k unique strings, execute the following code
        
        for i in range(0,len(allWords)-2):
            for j in range(i+1,len(allWords)-1):
                if (pd.isnull(allWords[i])): #If a string has no value (NaN), turn it into some nonsense
                    allWords[i] = "abcdef"
                elif (pd.isnull(allWords[j])):
                    allWords[j] = "abcdef"
            
                word_1 = wn.synsets(allWords[i])
                word_2 = wn.synsets(allWords[j])
            
                if (word_1 != [] and word_2 != []): #Calculate similarity between two non-empty words
                    similarity = wn.wup_similarity(word_1[0], word_2[0])
                    #print ("Similarity between",word_1[0],"and",word_2[0],":",similarity) #For debugging purposes
                    similarityScores.append(similarity)
        
        # Use this part of the code if you do not want to calculate a similarity score between
        # every single string, but only every string with one single other string.
        #
        #for i in range(0,len(allWords)-1):
        #    if (pd.isnull(allWords[i])):
        #        allWords[i] = "abcdef"
        #    elif (pd.isnull(allWords[i+1])):
        #        allWords[i+1] = "abcdef"
        #    
        #    word_1 = wn.synsets(allWords[i])
        #    word_2 = wn.synsets(allWords[i+1])
        #    
        #    if (word_1 != [] and word_2 != []):
        #        similarity = wn.wup_similarity(word_1[0], word_2[0])
        #        print ("Similarity between",word_1[0],"and",word_2[0],":",similarity)
        #        similarityScores.append(similarity)
    
        #print ("Similarity Scores:\n",similarityScores) #For debugging purposes
        print ("Mean Similarity Score:",np.mean(similarityScores))
    
        if (np.mean(similarityScores) > 0.70): #0.70 = Critical similarity value
            categorical = True
    
    print ("Categorical Feature?",categorical)
    
    return categorical

In [171]:
autoCheckCategoricalString(raw_data["country"],50)

Mean Similarity Score: 0.737473869897
Categorical Feature? True


True

In [172]:
def autoCheckCategoricalInt(raw_data_feature, k):
    categorical = False
    
    if (len(pd.unique(raw_data_feature)) <= k):
        categorical = True

    return categorical

In [182]:
def autoCheckCategoricals(raw_data,predicted,k):
    
    print ("Checking if any of the features has categorical data...")
    
    for j in range(0,len(predicted)):
        if (predicted[j] == 'int64'):
            if (autoCheckCategoricalInt(raw_data.iloc[:,j],k)):
                predicted[j] = 'cat_int64'
        elif (predicted[j] == 'string'):
            if (autoCheckCategoricalString(raw_data.iloc[:,j],k)):
                predicted[j] = 'cat_string'
                
    return predicted

In [183]:
predicted = autoCheckCategoricals(raw_data,predicted,50)
print ("\nPredicted data types:\n",predicted)

Checking if any of the features has categorical data...
Mean Similarity Score: 0.737473869897
Categorical Feature? True
Categorical Feature? False
Categorical Feature? False
Categorical Feature? False
Categorical Feature? False
Categorical Feature? True
Categorical Feature? False
Categorical Feature? False

Predicted data types:
 ['int64', 'cat_string', 'string', 'string', 'cat_int64', 'float64', 'string', 'string', 'cat_string', 'string', 'string']


In [117]:
#nltk.download('wordnet')

country_1 = wn.synsets(raw_data["country"][0])
country_2 = wn.synsets(raw_data["country"][1])
country_3 = wn.synsets(raw_data["country"][4])
country_1
country_2
country_3
province_1 = wn.synsets(raw_data["province"][0])
province_1
province_3 = wn.synsets(raw_data["province"][3])
province_3

#wn.synset('united_states.n.01').path_similarity(wn.synset('spain.n.01'))
wn.path_similarity(province_3[0],province_1[0])
wn.wup_similarity(province_3[0],province_1[0])

wn.synsets('turkey')

wn.wup_similarity(wn.synset('india.n.01'),wn.synset('turkey.n.05'))

[Synset('united_states.n.01'),
 Synset('uracil.n.01'),
 Synset('uranium.n.01'),
 Synset('u.n.03')]

[Synset('spain.n.01')]

[Synset('france.n.01'), Synset('france.n.02')]

[Synset('california.n.01')]

[Synset('oregon.n.01')]

0.3333333333333333

0.9

[Synset('turkey.n.01'),
 Synset('turkey.n.02'),
 Synset('joker.n.02'),
 Synset('turkey.n.04'),
 Synset('turkey.n.05')]

0.1111111111111111